<a href="https://colab.research.google.com/github/jvishnuvardhan/Keras_Examples/blob/master/TF2_bug_embedding_column.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip -q install tf-nightly-2.0-preview

     |████████████████████████████████| 90.6MB 3.1MB/s 
     |████████████████████████████████| 3.8MB 47.6MB/s 
     |████████████████████████████████| 460kB 52.3MB/s 


In [0]:
!python -c "import tensorflow as tf; print(tf.version.GIT_VERSION, tf.version.VERSION)"

v1.12.1-10538-gd526881 2.0.0-dev20190906


In [0]:
import sys
sys.version

'3.6.8 (default, Jan 14 2019, 11:02:34) \n[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]'

In [0]:
import tensorflow as tf
tf.__version__

'2.0.0-dev20190906'

In [0]:
import pandas as pd


def create_model():
    store_feature = tf.feature_column.categorical_column_with_vocabulary_list('store',
                                                                              vocabulary_list=['a', 'b'])

    store_feature = tf.feature_column.embedding_column(store_feature, dimension=64)

    loc_feature = tf.feature_column.categorical_column_with_vocabulary_list('loc',
                                                                              vocabulary_list=['x', 'y', 'z'])

    loc_feature = tf.feature_column.embedding_column(loc_feature, dimension=32)

    model = tf.keras.Sequential([
        tf.keras.layers.DenseFeatures([store_feature, loc_feature]),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='relu')
    ])

    return model

df = pd.DataFrame({'store': ['a', 'b', 'b', 'a'],
                   'loc': ['x', 'y', 'z', 'x'],
                   'label': [100., 200., 400., 100.]})


labels = df.pop('label')

ds = tf.data.Dataset.from_tensor_slices((dict(df), labels)).batch(2)

model_1 = create_model()
model_1.compile(optimizer='adam', loss='mse', metrics=['mse'])
model_1.fit(ds, epochs=5)
model_1.save_weights('./sample_chkpt-1')

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
2/2 [==============================] - 1s 403ms/step - loss: 54992.2881 - mse: 54992.2891
Epoch 2/5
2/2 [==============================] - 0s 6ms/step - loss: 39977.0015 - mse: 54974.2109
Epoch 3/5
2/2 [==============================] - 0s 5ms/step - loss: 39962.6724 - mse: 54957.4258
Epoch 4/5
2/2 [==============================] - 0s 5ms/step - loss: 39949.2603 - mse: 54941.2500
Epoch 5/5
2/2 [==============================] - 0s 5ms/step - loss: 39936.9004 - mse: 54926.6289


In [0]:
# this workds
model_1.predict(ds)

array([[0.        ],
       [0.31282616],
       [0.2581989 ],
       [0.        ]], dtype=float32)

In [0]:
model_2 = create_model()
model_2.load_weights('./sample_chkpt-1')

# this does not work
model_2.predict(ds)

array([[0.        ],
       [0.31282616],
       [0.2581989 ],
       [0.        ]], dtype=float32)